## Spark Context and Imports

In [1]:
import os, re, sys, utilities
spark_home = os.environ.get('SPARK_HOME', None)

In [2]:
import sys
sys.version_info

sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)

In [3]:
code = utilities.getSparkContext()
exec(code)

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2.3.1.5.80-6
      /_/

Using Python version 3.6.8 (default, Dec 30 2018 01:22:34)
SparkSession available as 'spark'.


In [4]:
from pyspark import SparkContext
#from pyspark.sql import SparkSession, HiveContext

In [5]:
from pyspark_llap.sql.session import HiveWarehouseSession
from pyspark.sql import SparkSession

hive = HiveWarehouseSession.session(spark).build()

In [6]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import datetime
import math
import scipy

In [7]:
from pyspark.sql.functions import col, udf, countDistinct, isnan, when, count, to_date, from_unixtime, unix_timestamp, min, max, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import Row
from pyspark.sql.functions import year, month, dayofmonth, hour

In [8]:
from pyspark.sql.types import DoubleType, FloatType, BooleanType, LongType, \
IntegerType, TimestampType, StringType, StructType, StructField, ArrayType

In [9]:
import pyspark.sql.functions as f

### Read in parquet file from HDFS - preprocessed EU data

In [10]:
# using SQLContext to read parquet file
#from pyspark.sql import SQLContext
#sqlContext = SQLContext(SparkContext)

# to read parquet file - EU Preprocessed
df = spark.read.parquet('hdfs://hdp2cluster/user/lguerra5/pae_processing_prod/trip_summary_preprocessed_subset/*')

In [11]:
df.show(5)

+------------------------+-----------------+-----------------+-------------+-----------+---------------+-----------------------+------------+--------------+-------------+
|trip_end_eng_oil_percent|trip_end_odo_read|              vin|trip_end_date|trip_number|oil_change_flag|oil_change_cycle_number|first_ol_pct|first_odometer|first_ol_date|
+------------------------+-----------------+-----------------+-------------+-----------+---------------+-----------------------+------------+--------------+-------------+
|                    97.0|            46770|1FA6P8CF8J5125374|   2019-09-25|       2455|              0|                      8|       100.0|         75915|   2020-08-25|
|                    97.0|            46781|1FA6P8CF8J5125374|   2019-09-25|       2456|              0|                      8|       100.0|         75915|   2020-08-25|
|                    97.0|            46793|1FA6P8CF8J5125374|   2019-09-25|       2457|              0|                      8|       100.0|    

In [10]:
#df.orderBy(["value", "rank"], ascending=[1, 1])
df.orderBy('trip_end_date', ascending=0).show(20)

+------------------------+-----------------+-----------------+-------------+-----------+---------------+-----------------------+------------+--------------+-------------+
|trip_end_eng_oil_percent|trip_end_odo_read|              vin|trip_end_date|trip_number|oil_change_flag|oil_change_cycle_number|first_ol_pct|first_odometer|first_ol_date|
+------------------------+-----------------+-----------------+-------------+-----------+---------------+-----------------------+------------+--------------+-------------+
|                    52.0|            22611|1FTEW1EG1JFC96919|   2020-05-18|       2487|              0|                      2|       100.0|         19317|   2019-12-20|
|                    38.0|            72626|1FTEW1EF4HKD62663|   2020-05-18|       4199|              0|                      5|       100.0|         62891|   2019-12-14|
|                    59.0|             5000|1FTEW1EP3KKF04730|   2020-05-18|        595|              0|                      0|       100.0|    

In [11]:
df.printSchema()

root
 |-- trip_end_date: date (nullable = true)
 |-- trip_end_eng_oil_percent: float (nullable = true)
 |-- trip_end_odo_read: long (nullable = true)
 |-- vin: string (nullable = true)
 |-- trip_number: integer (nullable = true)
 |-- oil_change_flag: integer (nullable = true)
 |-- oil_change_cycle_number: integer (nullable = true)
 |-- first_ol_pct: float (nullable = true)
 |-- first_odometer: long (nullable = true)
 |-- first_ol_date: date (nullable = true)



In [ ]:
#How many messages in preprocesed data?
prep_total_msg = df.count()

In [ ]:
print(prep_total_msg)

In [10]:
#how many distinct VINs in prep data?
total_eu_vins_prep = df.select("vin").distinct().count()
print(total_eu_vins_prep)

507053


In [ ]:
# Date range for both 

In [ ]:
# Preprocessed Data
max_val1 = df.agg({"trip_end_date": "max"}).collect()[0][0]
min_val1 = df.agg({"trip_end_date": "min"}).collect()[0][0]

In [ ]:
print(max_val1)
print(min_val1)

### For Eu data, compare mapping table to Prep output VINS

In [12]:
EU_vins = df.select("vin")

In [17]:
EU_vins.select('vin').distinct().show(20)

+-----------------+
|              vin|
+-----------------+
|6FPPXXMJ2PKS33137|
|6FPPXXMJ2PKS33767|
|6FPPXXMJ2PLB79488|
|6FPPXXMJ2PLR79340|
|WF01XXERK1JR51276|
|WF01XXERK1KD07553|
|WF01XXERK1KJ74862|
|WF01XXERK1KK89994|
|WF01XXERK1KL30849|
|WF01XXERK1KL33839|
|WF01XXERK1KS67232|
|WF03XXTTG3LB73920|
|WF0EXXTTREKJ43517|
|WF0EXXTTREKS04947|
|WF0EXXTTREKS26013|
|WF0EXXTTRELB76637|
|WF0FXXGAHFKE49489|
|WF0FXXWPMHLR78685|
|WF0JXXGAHJJA35459|
|WF0JXXGAHJKC65979|
+-----------------+
only showing top 20 rows



In [13]:
EU_vins = EU_vins.drop_duplicates(subset=['vin']) 

In [ ]:
EU_vins.printSchema()

In [38]:
EU_vins.show(20)

+-----------------+
|              vin|
+-----------------+
|6FPPXXMJ2PKU63902|
|6FPPXXMJ2PLR85406|
|6FPPXXMJ2PLR85771|
|WF01XXERK1JA56920|
|WF01XXERK1JG65130|
|WF01XXERK1JR43010|
|WF01XXERK1JR45246|
|WF01XXERK1KJ78990|
|WF01XXERK1KK86161|
|WF01XXERK1KL42524|
|WF01XXERK1KM07374|
|WF01XXERK1KY51968|
|WF01XXTTG1KM14958|
|WF01XXTTG1KP47141|
|WF01XXTTG1KT36429|
|WF03XXTTGHLR86898|
|WF0EXXTTREKS11254|
|WF0EXXTTRELB78449|
|WF0FXXGAHFKT42334|
|WF0FXXGAHFKY21492|
+-----------------+
only showing top 20 rows



In [14]:
# read csv file with limits mapping table
limits_map = spark.read.format("csv").option("header", "true").load('hdfs://hdp2cluster/user/lguerra5/EU_limitsmap.csv')

In [15]:
limits_map.show(5)

+-----------------+-----------------------+----------------+
|              VIN|UPPER_LIMIT_ODOMETER_KM|UPPER_LIMIT_DAYS|
+-----------------+-----------------------+----------------+
|WF02K8GY1LGA84967|                  20117|             365|
|WF02K8GY4LGA84896|                  20117|             365|
|WF02K8HY0LGA00281|                  20117|             365|
|WF02XXERK2KM10920|                  30000|             365|
|WF02XXERK2KM10922|                  15000|             365|
+-----------------+-----------------------+----------------+
only showing top 5 rows



In [16]:
map_join = limits_map.join(EU_vins, limits_map.VIN == EU_vins.vin, how='inner')

In [17]:
map_join.show(20)

+-----------------+-----------------------+----------------+-----------------+
|              VIN|UPPER_LIMIT_ODOMETER_KM|UPPER_LIMIT_DAYS|              vin|
+-----------------+-----------------------+----------------+-----------------+
|WF0NXXGCHNKU52453|                  30000|             730|WF0NXXGCHNKU52453|
|WF0PXXGCHPKT09586|                  20000|             365|WF0PXXGCHPKT09586|
|WF0NXXGCHNKM71530|                  30000|             730|WF0NXXGCHNKM71530|
|WF0NXXGCHNKP04903|                  30000|             730|WF0NXXGCHNKP04903|
|WF0NXXGCHNLR39724|                  30000|             730|WF0NXXGCHNLR39724|
|WF0NXXGCHNKT11548|                  20000|             365|WF0NXXGCHNKT11548|
|WF0NXXGCHNKM67115|                  30000|             730|WF0NXXGCHNKM67115|
|WF0NXXGCHNKT00448|                  20000|             365|WF0NXXGCHNKT00448|
|WF0NXXGCHNLB12991|                  30000|             730|WF0NXXGCHNLB12991|
|WF0NXXGCHNKL44715|                  20000|         

In [18]:
map_join.printSchema()

root
 |-- VIN: string (nullable = true)
 |-- UPPER_LIMIT_ODOMETER_KM: string (nullable = true)
 |-- UPPER_LIMIT_DAYS: string (nullable = true)
 |-- vin: string (nullable = true)



In [22]:
map_join[map_join['UPPER_LIMIT_ODOMETER_KM'] == '20000'].show(5)

+-----------------+-----------------------+----------------+-----------------+
|              VIN|UPPER_LIMIT_ODOMETER_KM|UPPER_LIMIT_DAYS|              vin|
+-----------------+-----------------------+----------------+-----------------+
|WF0NXXGCHNKT11548|                  20000|             365|WF0NXXGCHNKT11548|
|WF0NXXGCHNKU39552|                  20000|             365|WF0NXXGCHNKU39552|
|WF0NXXGCHNKL60657|                  20000|             365|WF0NXXGCHNKL60657|
|WF0NXXGCHNKT88244|                  20000|             365|WF0NXXGCHNKT88244|
|WF0NXXGCHNKL44715|                  20000|             365|WF0NXXGCHNKL44715|
+-----------------+-----------------------+----------------+-----------------+
only showing top 5 rows



In [29]:
map_join.groupBy("UPPER_LIMIT_ODOMETER_KM").count().show()

+-----------------------+-----+
|UPPER_LIMIT_ODOMETER_KM|count|
+-----------------------+-----+
|                  20117|    3|
|                  20100|   12|
|                  20000| 2610|
|                  30000| 3773|
|                  15000|    1|
+-----------------------+-----+



### Output data for E2E

In [24]:
test_case = df[(df.vin == '1FMCU0J93HUC30985')].toPandas()

In [28]:
test_case.shape

(1500, 10)

In [26]:
dup_columns = ['trip_end_odo_read', 'trip_end_date', 'trip_end_eng_oil_percent',
                                   'oil_change_cycle_number']

In [27]:
test_case = test_case.sort_values(by=['trip_end_odo_read', 'trip_end_date','trip_end_eng_oil_percent'],
                                                ascending=True).drop_duplicates(subset=dup_columns, keep='last')

In [29]:
test_case.to_csv("/s/lguerra5/oilreset_dd_1FMCU0J93HUC30985.csv")

### US Pre-processing read and grab one VIN

In [36]:
# to read parquet file - EU Preprocessed
df = spark.read.parquet('hdfs://hdp2cluster/user/lguerra5/pae_processing_prod/trip_summary_preprocessed_subset/*')

In [10]:
#df.orderBy(["value", "rank"], ascending=[1, 1])
df.show(20)

+------------------------+-----------------+-----------------+-------------+-----------+---------------+-----------------------+------------+--------------+-------------+
|trip_end_eng_oil_percent|trip_end_odo_read|              vin|trip_end_date|trip_number|oil_change_flag|oil_change_cycle_number|first_ol_pct|first_odometer|first_ol_date|
+------------------------+-----------------+-----------------+-------------+-----------+---------------+-----------------------+------------+--------------+-------------+
|                   100.0|               40|1FA6P8CF1K5176278|   2019-04-20|          0|              0|                      0|       100.0|          8424|   2019-07-24|
|                   100.0|               48|1FA6P8CF1K5176278|   2019-04-20|          1|              0|                      0|       100.0|          8424|   2019-07-24|
|                   100.0|              134|1FA6P8CF1K5176278|   2019-04-20|          2|              0|                      0|       100.0|    

In [11]:
vin_list = {'1FD0W4GT3KEF72655','1FD0W4GT5KEF10741','1FD0W4HT0KEF94966','1FD0W4HT1KED45673','1FD0W4HT1KEE51007','1FD0W4HT1KEE80085','1FD0W4HT1KEE83326', \
            '1FD0W4HT3KEC19671','1FD0W4HT3KED37784','1FD0W4HT3KEE79357','1FD0W4HT4KEE14842','1FD0W4HT4KEF60450','1FD0W4HT5KED98862','1FD0W4HT5KEE91977', \
            '1FD0W4HT5KEF75815','1FD0W4HT7KEC81395','1FD0W4HT7KEE46765','1FD0W4HT8KEC29340','1FD0W4HT8KEE73232','1FD0W4HT8KEF10800','1FD0W4HT9KEE46766', \
            '1FD0W4HT9KEE91979','1FD0W4HT9KEF13964','1FD0W4HT9KEF95548'}

In [14]:
df = df.where(f.col("vin").isin(vin_list))

In [37]:
vin_list_df = df[(df.vin == '1FMCU9J98HUA48350')]

In [38]:
#test_case = df[(df.vin == '1FD0W4GT3KEF72655')].toPandas()
vin_list_check = vin_list_df.toPandas()

In [39]:
dup_columns = ['trip_end_odo_read', 'trip_end_date', 'trip_end_eng_oil_percent','oil_change_cycle_number']
vin_list_check = vin_list_check.sort_values(by=['trip_end_odo_read', 'trip_end_date','trip_end_eng_oil_percent'],ascending=True).drop_duplicates(subset=dup_columns, keep='last')

In [40]:
vin_list_check.shape

(1500, 10)

In [41]:
vin_list_check.to_csv("/s/lguerra5/trip_sum_1FMCU9J98HUA48350.csv")

### Ad Hoc request - grab full trip history for a few VINS

#### Get data from trip summary

In [10]:
hiveQuery = "SELECT T1.cvdcqa_vin_d_3 AS vin, \
coalesce(T1.cvdcqa_oil_chng_cyc_no_r_3,0) AS oil_change_cycle_number, \
T1.cvdcqa_trip_no_r_3 AS trip_number, \
T1.cvdcqa_trip_end_time_s_3 AS trip_end_time, \
T1.cvdcqa_trip_end_eng_oil_p_3 AS trip_end_eng_oil_percent, \
T1.cvdcqa_trip_end_odo_read_r_3 AS trip_end_odo_read \
FROM cvdp.NCVDVCJ_TRIP_SUM_FINAL_4G_NA_USA_MSI_VW T1 \
WHERE T1.cvdcqa_suspect_flag_r_3 = 0 AND \
T1.cvdcqa_trip_no_r_3 IS NOT NULL AND \
T1.cvdcqa_trip_end_time_s_3 IS NOT NULL AND \
T1.cvdcqa_trip_end_eng_oil_p_3 IS NOT NULL AND \
T1.cvdcqa_trip_end_odo_read_r_3 IS NOT NULL AND \
T1.cvdcqa_vin_d_3 IN ('1FMCU9J98HUA48350', 'MAJ3P1TE2JC209476', '5LMCJ1C96KUL16887') AND \
T1.cvdcqa_partition_cntry_x = 'USA' AND \
year(T1.cvdcqa_trip_end_time_s_3) >= '2017'"

In [11]:
# new table name: NCVDVCJ_TRIP_SUM_FINAL_4G_NA_USA_MSI_VW

In [12]:
df_tsm = hive.executeQuery(hiveQuery)

In [13]:
df_tsm.dtypes

[('vin', 'string'),
 ('oil_change_cycle_number', 'int'),
 ('trip_number', 'int'),
 ('trip_end_time', 'timestamp'),
 ('trip_end_eng_oil_percent', 'double'),
 ('trip_end_odo_read', 'int')]

In [17]:
df_tsm.show(5)

+-----------------+-----------------------+-----------+-------------------+------------------------+-----------------+
|              vin|oil_change_cycle_number|trip_number|      trip_end_time|trip_end_eng_oil_percent|trip_end_odo_read|
+-----------------+-----------------------+-----------+-------------------+------------------------+-----------------+
|1FTEW1EG1KFC26175|                      0|        453|2020-03-07 03:40:19|                    63.0|             6030|
|1FTEW1EG1KFC26175|                      0|        454|2020-03-07 04:17:43|                    63.0|             6033|
|1FTEW1EG1KFC26175|                      0|        455|2020-03-07 04:17:46|                    63.0|             6033|
|1FTEW1EG1KFC26175|                      0|        456|2020-03-07 04:35:39|                    63.0|             6038|
|1FMCU0J93HUC30985|                      4|       2896|2020-09-20 17:01:39|                     6.0|            73267|
+-----------------+-----------------------+-----

In [15]:
df_tsm = df_tsm.withColumn('trip_end_date', from_unixtime(unix_timestamp(col('trip_end_time'),"yyyy, MM, dd, HH, mm, ss"),"yyyy-MM-dd"))

In [16]:
df_tsm.show(5)

+-----------------+-----------------------+-----------+-------------------+------------------------+-----------------+-------------+
|              vin|oil_change_cycle_number|trip_number|      trip_end_time|trip_end_eng_oil_percent|trip_end_odo_read|trip_end_date|
+-----------------+-----------------------+-----------+-------------------+------------------------+-----------------+-------------+
|MAJ3P1TE2JC209476|                      2|       1855|2019-10-23 19:59:17|                    94.0|            17424|   2019-10-23|
|MAJ3P1TE2JC209476|                      2|       1854|2019-10-23 09:28:01|                    94.0|            17411|   2019-10-23|
|MAJ3P1TE2JC209476|                      2|       1853|2019-10-23 09:09:52|                    94.0|            17404|   2019-10-23|
|MAJ3P1TE2JC209476|                      2|       2329|2020-03-06 19:09:18|                    57.0|            21220|   2020-03-06|
|MAJ3P1TE2JC209476|                      2|       2330|2020-03-06 19:

In [ ]:
# filter in spark

In [25]:
df1 = df_tsm[(df_tsm.vin == '1FMCU9J98HUA48350')]

In [26]:
dup_columns = ['trip_end_odo_read', 'trip_end_date', 'trip_end_eng_oil_percent','oil_change_cycle_number']
df1 = df_tsm.orderBy(dup_columns, ascending = True).dropDuplicates(subset = dup_columns)

In [27]:
tc1 = df1.toPandas()

In [28]:
tc1.to_csv("/s/lguerra5/full_hist_1FMCU9J98HUA48350.csv")

In [ ]:
# filter and sort in pandas

In [29]:
test_case = df_tsm[(df_tsm.vin == '5LMCJ1C96KUL16887')].toPandas()

In [30]:
dup_columns = ['trip_end_odo_read', 'trip_end_date', 'trip_end_eng_oil_percent','oil_change_cycle_number']

In [31]:
test_case = test_case.sort_values(by=['trip_end_odo_read', 'trip_end_date','trip_end_eng_oil_percent'], ascending=True).drop_duplicates(subset=dup_columns, keep='last')

In [32]:
test_case.shape

(1392, 7)

In [33]:
test_case.to_csv("/s/lguerra5/full_hist_5LMCJ1C96KUL16887.csv")

## PAE Early Prediction Cases:
### Get recent data from VINs in Andew's examples

In [10]:
hiveQuery = "SELECT T1.cvdcqa_vin_d_3 AS vin, \
coalesce(T1.cvdcqa_oil_chng_cyc_no_r_3,0) AS oil_change_cycle_number, \
T1.cvdcqa_trip_no_r_3 AS trip_number, \
T1.cvdcqa_trip_end_time_s_3 AS trip_end_time, \
T1.cvdcqa_trip_end_eng_oil_p_3 AS trip_end_eng_oil_percent, \
T1.cvdcqa_trip_end_odo_read_r_3 AS trip_end_odo_read \
FROM cvdp.NCVDVCJ_TRIP_SUM_FINAL_4G_NA_USA_MSI_VW T1 \
WHERE T1.cvdcqa_suspect_flag_r_3 = 0 AND \
T1.cvdcqa_trip_no_r_3 IS NOT NULL AND \
T1.cvdcqa_trip_end_time_s_3 IS NOT NULL AND \
T1.cvdcqa_trip_end_eng_oil_p_3 IS NOT NULL AND \
T1.cvdcqa_trip_end_odo_read_r_3 IS NOT NULL AND \
T1.cvdcqa_vin_d_3 IN ('1FMCU0J93HUC30985', '1FTEW1EG1KFC26175', '1FTER4EH3KLA94443') AND \
T1.cvdcqa_partition_cntry_x = 'USA' AND \
year(T1.cvdcqa_trip_end_time_s_3) >= '2020'"

In [ ]:
vin_list ={'1FMCU9HD9KUC01559','2FMPK4AP2KBC13976','1FMCU9GD0KUC55902','1FMCU9GD5KUC56107','3FA6P0LU1KR268780','1FMCU0J93HUC30985', \
           '1FMSK7FH9LGA14639','3FA6P0HD0LR119383','2FMPK4AP3KBB70085','2LMPJ8LP7KBL62041','1FTEW1EG1KFC26175','MAJ6S3GL6KC267026', \
           '5LM5J9XC3LGL02491','1FMCU9GD0HUB88078','1FTEW1E50KKC55110','1FTER4EH3KLA94443'}


In [11]:
df_tsm = hive.executeQuery(hiveQuery)

In [12]:
df_tsm.dtypes

[('vin', 'string'),
 ('oil_change_cycle_number', 'int'),
 ('trip_number', 'int'),
 ('trip_end_time', 'timestamp'),
 ('trip_end_eng_oil_percent', 'double'),
 ('trip_end_odo_read', 'int')]

In [13]:
df_tsm.show(5)

+-----------------+-----------------------+-----------+-------------------+------------------------+-----------------+
|              vin|oil_change_cycle_number|trip_number|      trip_end_time|trip_end_eng_oil_percent|trip_end_odo_read|
+-----------------+-----------------------+-----------+-------------------+------------------------+-----------------+
|1FTER4EH3KLA94443|                      0|        463|2020-03-13 11:27:20|                    65.0|             1755|
|1FTER4EH3KLA94443|                      0|        464|2020-03-13 16:30:39|                    65.0|             1764|
|1FTER4EH3KLA94443|                      0|        465|2020-03-13 16:31:22|                    65.0|             1764|
|1FMCU0J93HUC30985|                      4|       2905|2020-09-24 12:48:48|                     5.0|            73340|
|1FMCU0J93HUC30985|                      4|       2906|2020-09-24 13:14:49|                     5.0|            73342|
+-----------------+-----------------------+-----

In [14]:
df_tsm = df_tsm.withColumn('trip_end_date', from_unixtime(unix_timestamp(col('trip_end_time'),"yyyy, MM, dd, HH, mm, ss"),"yyyy-MM-dd"))

In [15]:
df_tsm.show(5)

+-----------------+-----------------------+-----------+-------------------+------------------------+-----------------+-------------+
|              vin|oil_change_cycle_number|trip_number|      trip_end_time|trip_end_eng_oil_percent|trip_end_odo_read|trip_end_date|
+-----------------+-----------------------+-----------+-------------------+------------------------+-----------------+-------------+
|1FMCU0J93HUC30985|                      4|       2905|2020-09-24 12:48:48|                     5.0|            73340|   2020-09-24|
|1FMCU0J93HUC30985|                      4|       2906|2020-09-24 13:14:49|                     5.0|            73342|   2020-09-24|
|1FMCU0J93HUC30985|                      4|       2907|2020-09-24 13:24:41|                     5.0|            73344|   2020-09-24|
|1FMCU0J93HUC30985|                      4|       2908|2020-09-24 13:24:44|                     5.0|            73344|   2020-09-24|
|1FMCU0J93HUC30985|                      4|       2909|2020-09-24 14:

In [16]:
test_case = df_tsm.toPandas()

In [15]:
#dup_columns = ['vin', 'date', 'trip_dur']
#df_tsm = df_tsm.orderBy(dup_columns, ascending = True).dropDuplicates(subset = dup_columns)

In [17]:
test_case.shape

(2252, 7)

In [36]:
dup_columns = ['trip_end_odo_read', 'trip_end_date', 'trip_end_eng_oil_percent','oil_change_cycle_number']

In [37]:
test_case = test_case.sort_values(by=['trip_end_odo_read', 'trip_end_date','trip_end_eng_oil_percent'], ascending=True).drop_duplicates(subset=dup_columns, keep='last')

In [18]:
test_case.to_csv("/s/lguerra5/ep_test.csv")